In [5]:
from scipy.stats import zscore
import numpy as np
import pandas as pd
from scipy import stats
import warnings
from statsmodels.stats.anova import AnovaRM
from scipy.stats import chi2_contingency
warnings.filterwarnings('ignore')


C:\Users\Aredio\AppData\Local\Temp\ipykernel_18600\1166996338.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Question 1

### t-test Time

In [10]:

data  =  pd.read_csv("answers.csv")
# Filter data for Question 1
data_q1 = data[data['Question Number'] == 1]

# Convert "Time Taken" to seconds
data_q1['Time Taken Seconds'] = data_q1['Time Taken'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

# Group by Condition and calculate mean time taken for each condition
mean_times_participtants = data_q1.groupby(['User ID','Condition'])['Time Taken Seconds'].mean().reset_index()
mean_times = mean_times_participtants.groupby('Condition')['Time Taken Seconds'].mean()
# Display the mean times for verification before proceeding to the Z-Test
mean_times

Condition
D    23.904762
J    33.952381
S    27.015873
Name: Time Taken Seconds, dtype: float64

In [11]:
from scipy.stats import ttest_ind

# Function to perform T-Test between two groups
def perform_ttest(group1, group2):
    t_stat, p_value = ttest_ind(group1, group2)
    return t_stat, p_value

# Extracting time taken seconds for each condition
times_d = data_q1[data_q1['Condition'] == 'D']['Time Taken Seconds']
times_j = data_q1[data_q1['Condition'] == 'J']['Time Taken Seconds']
times_s = data_q1[data_q1['Condition'] == 'S']['Time Taken Seconds']

# Perform pairwise T-Tests
t_stat_d_j, p_value_d_j = perform_ttest(times_d, times_j)
t_stat_d_s, p_value_d_s = perform_ttest(times_d, times_s)
t_stat_j_s, p_value_j_s = perform_ttest(times_j, times_s)

# Results
results = {
    "D vs J": {"T-Statistic": t_stat_d_j, "P-Value": p_value_d_j},
    "D vs S": {"T-Statistic": t_stat_d_s, "P-Value": p_value_d_s},
    "J vs S": {"T-Statistic": t_stat_j_s, "P-Value": p_value_j_s}
}

results


{'D vs J': {'T-Statistic': -2.541347708721478,
  'P-Value': 0.012275412460703973},
 'D vs S': {'T-Statistic': -0.9969956974188365, 'P-Value': 0.3207081840813019},
 'J vs S': {'T-Statistic': 1.663984951258184, 'P-Value': 0.09864061379810109}}

### Anova time

In [78]:
# Convert 'Time Taken' from 'mm:ss' to total seconds
data['Time Seconds'] = data['Time Taken'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

# Filter data for Question 1 only
data_q1 = data[data['Question Number'] == 1]

# Calculate the mean time taken for each condition across repetitions
mean_times = data_q1.groupby(['Condition', 'Repetition'])['Time Seconds'].mean().reset_index()

# Perform the ANOVA with repeated measures
anova_results = AnovaRM(data=mean_times, depvar='Time Seconds', subject='Repetition', within=['Condition']).fit()

print(anova_results)

                Anova
          F Value Num DF Den DF Pr > F
--------------------------------------
Condition 17.5186 2.0000 4.0000 0.0105



    F Value (17.5186): This value indicates the ratio of the variance between the groups to the variance within the groups. A higher F value suggests a greater difference between the means of the groups relative to the variation within the groups.

    Num DF (2.0000): This is the degrees of freedom for the numerator, corresponding to the number of groups (conditions) minus one. It indicates there are three conditions being compared.

    Den DF (4.0000): This is the degrees of freedom for the denominator, representing the total degrees of freedom within the groups. This value can be influenced by the total number of observations and the structure of the repeated measures.

    Pr > F (0.0105): This is the p-value, which measures the probability of observing the test results under the null hypothesis that there is no effect (no difference in means across the conditions). A p-value of 0.0105 indicates there is a statistically significant difference between the conditions at a conventional alpha level of 0.05. In other words, it's unlikely that these differences in time taken for each condition occurred by chance.

Given this result, you can conclude that the condition significantly affects the time taken to answer Question 1, and therefore, it would be worthwhile to further investigate the nature of these differences, possibly through post-hoc tests or additional analysis to understand which specific conditions differ from each other.

### t-Test Accuracy

In [7]:
# Filter data for question 1
question_1_data = data[data['Question Number'] == 1]

# Create contingency table
contingency_table_q1 = pd.crosstab(index=question_1_data['Condition'], columns=question_1_data['Correct'])

# Perform Chi-square test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table_q1)
print(f"Chi-square Statistic: {chi2_stat}, P-value: {p_value}")

Chi-square Statistic: 1.3375068045726726, P-value: 0.5123468702940044


### Anova accuracy

In [43]:
# Create a contingency table of correct responses by condition
contingency_table = pd.crosstab(data_q1['Condition'], data_q1['Correct'])

# Perform the Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi2 value: {chi2}, P-value: {p}, Degrees of Freedom: {dof}")
print("Expected frequencies if no difference in accuracy across conditions:")
print(expected)

Chi2 value: 1.3375068045726726, P-value: 0.5123468702940044, Degrees of Freedom: 2
Expected frequencies if no difference in accuracy across conditions:
[[ 7.33333333 55.66666667]
 [ 7.33333333 55.66666667]
 [ 7.33333333 55.66666667]]


Chi2 value: 1.3375, which is relatively low, indicating that the observed frequencies of correct and incorrect responses are not significantly different from what would be expected if there was no association between condition and accuracy.

P-value: 0.5123, which is much higher than the conventional alpha level of 0.05. This means there is not enough evidence to reject the null hypothesis, suggesting that there is no statistically significant difference in accuracy across the conditions for Question 1.

Degrees of Freedom (dof): 2, corresponding to the number of conditions minus one. This matches the setup of your experiment with three conditions.

Expected frequencies: The expected frequencies under the null hypothesis are fairly evenly distributed across the conditions, further indicating that any observed differences in correct response rates are likely due to chance

---

# Question 2

### t-test time

In [13]:
# Filter data for Question 1
data_q2 = data[data['Question Number'] == 2]

# Convert "Time Taken" to seconds
data_q2['Time Taken Seconds'] = data_q2['Time Taken'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

# Group by Condition and calculate mean time taken for each condition
mean_times = data_q2.groupby('Condition')['Time Taken Seconds'].mean().reset_index()

# Display the mean times for verification before proceeding to the Z-Test
mean_times

Condition  Time Taken Seconds
0         D           24.555556
1         J           27.857143
2         S           26.936508

In [14]:
from scipy.stats import ttest_ind

# Function to perform T-Test between two groups
def perform_ttest(group1, group2):
    t_stat, p_value = ttest_ind(group1, group2)
    return t_stat, p_value

# Extracting time taken seconds for each condition
times_d = data_q2[data_q2['Condition'] == 'D']['Time Taken Seconds']
times_j = data_q2[data_q2['Condition'] == 'J']['Time Taken Seconds']
times_s = data_q2[data_q2['Condition'] == 'S']['Time Taken Seconds']

# Perform pairwise T-Tests
t_stat_d_j, p_value_d_j = perform_ttest(times_d, times_j)
t_stat_d_s, p_value_d_s = perform_ttest(times_d, times_s)
t_stat_j_s, p_value_j_s = perform_ttest(times_j, times_s)

# Results
results = {
    "D vs J": {"T-Statistic": t_stat_d_j, "P-Value": p_value_d_j},
    "D vs S": {"T-Statistic": t_stat_d_s, "P-Value": p_value_d_s},
    "J vs S": {"T-Statistic": t_stat_j_s, "P-Value": p_value_j_s}
}

results


{'D vs J': {'T-Statistic': -0.8542521462523445, 'P-Value': 0.394612641513861},
 'D vs S': {'T-Statistic': -0.6011476975487624, 'P-Value': 0.5488389649597504},
 'J vs S': {'T-Statistic': 0.20818714005208983, 'P-Value': 0.8354244126113349}}

For all pairwise comparisons (D vs. J, D vs. S, and J vs. S) for Question 2, the P-Values are significantly higher than the standard alpha level of 0.05. This means that there is no statistically significant evidence to suggest differences between the conditions regarding the aspect tested (whether it be mean time taken, proportions of correct/incorrect answers, or any other metric). Therefore, we conclude that any observed differences are likely due to random variation rather than a systematic effect of the conditions.

---

### Anova time

In [88]:
# Convert 'Time Taken' from 'mm:ss' to total seconds
data['Time Seconds'] = data['Time Taken'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

# Filter data for Question 1 only
data_q2 = data[data['Question Number'] == 2]

# Calculate the mean time taken for each condition across repetitions
mean_times = data_q2.groupby(['Condition', 'Repetition'])['Time Seconds'].mean().reset_index()

# Perform the ANOVA with repeated measures
anova_results = AnovaRM(data=mean_times, depvar='Time Seconds', subject='Repetition', within=['Condition']).fit()

print(anova_results)

                Anova
          F Value Num DF Den DF Pr > F
--------------------------------------
Condition  0.7943 2.0000 4.0000 0.5123



### t-test Accuracy

In [21]:
df_q2 = data[data['Question Number'] == 2]
df_q2['Correct'] = pd.to_numeric(df_q2['Correct'], errors='coerce')
mean_correct_per_condition = df_q2.groupby('Condition')['Correct'].mean().reset_index()
# Extract the 'Correct' values for each condition
correct_d = df_q2[df_q2['Condition'] == 'D']['Correct']
correct_j = df_q2[df_q2['Condition'] == 'J']['Correct']
correct_s = df_q2[df_q2['Condition'] == 'S']['Correct']

# Conduct pairwise t-tests
ttest_d_j = ttest_ind(correct_d, correct_j, equal_var=False)
ttest_d_s = ttest_ind(correct_d, correct_s, equal_var=False)
ttest_j_s = ttest_ind(correct_j, correct_s, equal_var=False)

ttest_d_j, ttest_d_s, ttest_j_s

(TtestResult(statistic=0.7416437730881292, pvalue=0.45970564124261337, df=123.99244284029128),
 TtestResult(statistic=0.8588407317492376, pvalue=0.39209182381499985, df=123.53326329587462),
 TtestResult(statistic=0.1436633140468696, pvalue=0.8860006680826324, df=123.40834120044823))

    Condition D: Mean = 1.714286
    Condition J: Mean = 1.476190
    Condition S: Mean = 1.428571

Next, we will conduct pairwise t-tests between each condition to determine if there are any significant differences in accuracy. We will compare:

    Condition D vs. Condition J
    Condition D vs. Condition S
    Condition J vs. Condition S

Let's proceed with the t-tests. ​

​

The results of the pairwise t-tests for Question 2 are as follows:

    Condition D vs. Condition J: The t-statistic is 0.742 with a p-value of 0.460. This indicates that there is no significant difference in accuracy between conditions D and J.
    Condition D vs. Condition S: The t-statistic is 0.859 with a p-value of 0.392. This also indicates that there is no significant difference in accuracy between conditions D and S.
    Condition J vs. Condition S: The t-statistic is 0.144 with a p-value of 0.886. This further indicates that there is no significant difference in accuracy between conditions J and S.

---


### Anova accuracy

In [95]:
df_q2 = df[df['Question Number'] == 2].copy()

# Convert "Correct" to numerical values if necessary
# Assuming "Correct" in Question 2 already represents numerical accuracy values
df_q2['Correct'] = pd.to_numeric(df_q2['Correct'], errors='coerce')

# Calculate the average accuracy for each condition for each participant
avg_accuracy_q2 = df_q2.groupby(['User ID', 'Condition'])['Correct'].mean().reset_index()

# Prepare data for ANOVA repeated measures
anova_data = avg_accuracy_q2.rename(columns={'User ID': 'Subject', 'Condition': 'Within', 'Correct': 'Dependent'})

# Perform ANOVA repeated measures
anova_rm = AnovaRM(data=anova_data, depvar='Dependent', subject='Subject', within=['Within']).fit()

# Display the ANOVA table
print(anova_rm.summary())

               Anova
       F Value Num DF  Den DF Pr > F
------------------------------------
Within  0.2754 2.0000 40.0000 0.7607



: 

# Question 3

### t-test Time

In [79]:

data  =  pd.read_csv("answers.csv")
# Filter data for Question 1
data_q3 = data[data['Question Number'] == 3]

# Convert "Time Taken" to seconds
data_q3['Time Taken Seconds'] = data_q3['Time Taken'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

# Group by Condition and calculate mean time taken for each condition
mean_times = data_q3.groupby('Condition')['Time Taken Seconds'].mean().reset_index()

# Display the mean times for verification before proceeding to the Z-Test
mean_times


Condition  Time Taken Seconds
0         D           29.174603
1         J           23.174603
2         S           24.571429

In [80]:
from scipy.stats import ttest_ind

# Function to perform T-Test between two groups
def perform_ttest(group1, group2):
    t_stat, p_value = ttest_ind(group1, group2)
    return t_stat, p_value

# Extracting time taken seconds for each condition
times_d = data_q3[data_q3['Condition'] == 'D']['Time Taken Seconds']
times_j = data_q3[data_q3['Condition'] == 'J']['Time Taken Seconds']
times_s = data_q3[data_q3['Condition'] == 'S']['Time Taken Seconds']

# Perform pairwise T-Tests
t_stat_d_j, p_value_d_j = perform_ttest(times_d, times_j)
t_stat_d_s, p_value_d_s = perform_ttest(times_d, times_s)
t_stat_j_s, p_value_j_s = perform_ttest(times_j, times_s)

# Results
results = {
    "D vs J": {"T-Statistic": t_stat_d_j, "P-Value": p_value_d_j},
    "D vs S": {"T-Statistic": t_stat_d_s, "P-Value": p_value_d_s},
    "J vs S": {"T-Statistic": t_stat_j_s, "P-Value": p_value_j_s}
}

results


{'D vs J': {'T-Statistic': 1.4955536577884048, 'P-Value': 0.13731075083619707},
 'D vs S': {'T-Statistic': 1.075847668823263, 'P-Value': 0.2840841898318466},
 'J vs S': {'T-Statistic': -0.4735242143974047, 'P-Value': 0.6366718343191415}}

### Anova time

In [84]:
# Convert 'Time Taken' from 'mm:ss' to total seconds
data['Time Seconds'] = data['Time Taken'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

# Filter data for Question 1 only
data_q3 = data[data['Question Number'] == 3]

# Calculate the mean time taken for each condition across repetitions
mean_times = data_q3.groupby(['Condition', 'Repetition'])['Time Seconds'].mean().reset_index()

# Perform the ANOVA with repeated measures
anova_results = AnovaRM(data=mean_times, depvar='Time Seconds', subject='Repetition', within=['Condition']).fit()

print(anova_results)

                Anova
          F Value Num DF Den DF Pr > F
--------------------------------------
Condition 35.1705 2.0000 4.0000 0.0029



### t-test accuracy

In [13]:
# Filter data for question 1
question_3_data = data[data['Question Number'] == 3]

# Create contingency table
contingency_table_q3 = pd.crosstab(index=question_3_data['Condition'], columns=question_3_data['Correct'])

# Perform Chi-square test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table_q1)
print(f"Chi-square Statistic: {chi2_stat}, P-value: {p_value}")
print(question_3_data.groupby(['Condition','Count'])['Correct'].count())

Chi-square Statistic: 5.479881656804734, P-value: 0.06457416773679547
Condition
D    63
J    63
S    63
Name: Correct, dtype: int64


### Anova Accuracy

In [86]:
# Create a contingency table of correct responses by condition
contingency_table = pd.crosstab(data_q3['Condition'], data_q3['Correct'])

# Perform the Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi2 value: {chi2}, P-value: {p}, Degrees of Freedom: {dof}")
print("Expected frequencies if no difference in accuracy across conditions:")
print(expected)

Chi2 value: 5.479881656804734, P-value: 0.06457416773679547, Degrees of Freedom: 2
Expected frequencies if no difference in accuracy across conditions:
[[ 6.66666667 56.33333333]
 [ 6.66666667 56.33333333]
 [ 6.66666667 56.33333333]]
